In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,round

In [3]:
spark = SparkSession.builder.appName("Dataset_Description").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 09:38:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Competitions

In [5]:
competitions = spark.read.csv('/home/aniss/xG-model/Data/competitions.csv', header=True, inferSchema=True)

In [6]:
competitions.printSchema()

root
 |-- competition_id: integer (nullable = true)
 |-- season_id: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- competition_name: string (nullable = true)
 |-- competition_international: boolean (nullable = true)
 |-- season_name: string (nullable = true)



In [26]:
# Number of features and rows
len(competitions.columns), competitions.count()

(6, 26)

In [58]:
competitions.show(5,False)

+--------------+---------+------------------------+----------------------+-------------------------+-----------+
|competition_id|season_id|country_name            |competition_name      |competition_international|season_name|
+--------------+---------+------------------------+----------------------+-------------------------+-----------+
|223           |282      |South America           |Copa America          |true                     |2024       |
|55            |282      |Europe                  |UEFA Euro             |true                     |2024       |
|9             |281      |Germany                 |1. Bundesliga         |false                    |2023/2024  |
|44            |107      |United States of America|Major League Soccer   |false                    |2023       |
|1267          |107      |Africa                  |African Cup of Nations|true                     |2023       |
+--------------+---------+------------------------+----------------------+----------------------

In [60]:
competitions.groupBy('country_name','competition_name').count().orderBy('country_name').show(20,False)

+------------------------+----------------------+-----+
|country_name            |competition_name      |count|
+------------------------+----------------------+-----+
|Africa                  |African Cup of Nations|1    |
|England                 |Premier League        |1    |
|Europe                  |Champions League      |5    |
|Europe                  |UEFA Euro             |2    |
|France                  |Ligue 1               |3    |
|Germany                 |1. Bundesliga         |2    |
|International           |FIFA World Cup        |2    |
|Italy                   |Serie A               |1    |
|South America           |Copa America          |1    |
|Spain                   |La Liga               |7    |
|United States of America|Major League Soccer   |1    |
+------------------------+----------------------+-----+



### Matches

In [28]:
matches = spark.read.csv('/home/aniss/xG-model/Data/matches.csv', header=True, inferSchema=True)

In [29]:
matches.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- match_date: date (nullable = true)
 |-- competition: string (nullable = true)
 |-- season: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- match_week: integer (nullable = true)
 |-- competition_stage: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- home_managers: string (nullable = true)
 |-- away_managers: string (nullable = true)



In [30]:
# Number of features and rows
len(matches.columns), matches.count()

(14, 2450)

In [31]:
matches.show(5,truncate=False)

+--------+----------+----------------------------+------+---------+---------+----------+----------+----------+-----------------+-----------------------+-----------------------------+------------------------------+------------------------------+
|match_id|match_date|competition                 |season|home_team|away_team|home_score|away_score|match_week|competition_stage|stadium                |referee                      |home_managers                 |away_managers                 |
+--------+----------+----------------------------+------+---------+---------+----------+----------+----------+-----------------+-----------------------+-----------------------------+------------------------------+------------------------------+
|3943077 |2024-07-15|South America - Copa America|2024  |Argentina|Colombia |1         |0         |6         |Final            |Hard Rock Stadium      |Raphael Claus                |Lionel Sebastián Scaloni      |Néstor Gabriel Lorenzo        |
|3943076 |2024-07-14

In [61]:
# Number of matches per competition
matches.groupBy('competition').count().orderBy('count',ascending=False).show(28,False)

+----------------------------------------------+-----+
|competition                                   |count|
+----------------------------------------------+-----+
|Spain - La Liga                               |590  |
|France - Ligue 1                              |435  |
|Italy - Serie A                               |380  |
|England - Premier League                      |380  |
|Germany - 1. Bundesliga                       |340  |
|International - FIFA World Cup                |128  |
|Europe - UEFA Euro                            |102  |
|Africa - African Cup of Nations               |52   |
|South America - Copa America                  |32   |
|United States of America - Major League Soccer|6    |
|Europe - Champions League                     |5    |
+----------------------------------------------+-----+



In [33]:
matches.groupBy().avg('home_score','away_score') \
    .withColumn('home_score',round(col('avg(home_score)'),3)) \
    .withColumn('away_score',round(col('avg(away_score)'),3)) \
    .drop('avg(home_score)','avg(away_score)') \
.show()

+----------+----------+
|home_score|away_score|
+----------+----------+
|     1.539|     1.198|
+----------+----------+



### Lineups

In [34]:
lineups = spark.read.csv('/home/aniss/xG-model/Data/lineups.csv', header=True, inferSchema=True)

In [35]:
lineups.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_nickname: string (nullable = true)
 |-- jersey_number: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- cards: string (nullable = true)
 |-- positions: string (nullable = true)



In [36]:
# Number of features and rows
len(lineups.columns), lineups.count()

(7, 95630)

In [37]:
lineups.show(5,truncate=False)

+---------+-------------------------------+------------------+-------------+---------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|player_id|player_name                    |player_nickname   |jersey_number|country  |cards                                                                                   |positions                                    

### Events


In [40]:
events = spark.read.csv('/home/aniss/xG-model/Data/events.csv', header=True, inferSchema=True, sep=';')

In [41]:
events.printSchema()

root
 |-- 50_50: string (nullable = true)
 |-- ball_receipt_outcome: string (nullable = true)
 |-- ball_recovery_recovery_failure: boolean (nullable = true)
 |-- carry_end_location: string (nullable = true)
 |-- clearance_aerial_won: boolean (nullable = true)
 |-- counterpress: boolean (nullable = true)
 |-- dribble_nutmeg: boolean (nullable = true)
 |-- dribble_outcome: string (nullable = true)
 |-- dribble_overrun: boolean (nullable = true)
 |-- duel_outcome: string (nullable = true)
 |-- duel_type: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- foul_committed_advantage: boolean (nullable = true)
 |-- foul_committed_card: string (nullable = true)
 |-- foul_won_advantage: boolean (nullable = true)
 |-- foul_won_defensive: boolean (nullable = true)
 |-- goalkeeper_body_part: string (nullable = true)
 |-- goalkeeper_end_location: string (nullable = true)
 |-- goalkeeper_outcome: string (nullable = true)
 |-- goalkeeper_position: string (nullable = true)
 |-- goalk

In [42]:
# Number of features and rows
len(events.columns), events.count()

(122, 8710714)

In [66]:
ex = ['id','index','match_id','minute','second','period','play_pattern','possession',
      'possession_team','possession_team_id','related_events','team','team_id','timestamp','type','competition','season']
# select random rows
events.filter(events.type=='Shot').select(ex).show(5,truncate=False)

+------------------------------------+-----+--------+------+------+------+-------------+----------+---------------+------------------+--------------------------------------------------------------------------------+---------+-------+-----------------------+----+----------------------------+------+
|id                                  |index|match_id|minute|second|period|play_pattern |possession|possession_team|possession_team_id|related_events                                                                  |team     |team_id|timestamp              |type|competition                 |season|
+------------------------------------+-----+--------+------+------+------+-------------+----------+---------------+------------------+--------------------------------------------------------------------------------+---------+-------+-----------------------+----+----------------------------+------+
|3c3fd192-a115-47bc-9224-b5011bb17885|37   |3943077 |0     |36    |1     |From Kick Off|2         |Arge

### frames

In [54]:
frames = spark.read.csv('/home/aniss/xG-model/Data/frames.csv', header=True, inferSchema=True)

In [55]:
frames.printSchema()

root
 |-- id: string (nullable = true)
 |-- visible_area: string (nullable = true)
 |-- match_id: integer (nullable = true)
 |-- teammate: boolean (nullable = true)
 |-- actor: boolean (nullable = true)
 |-- keeper: boolean (nullable = true)
 |-- location: string (nullable = true)



In [56]:
# Number of features and rows
len(frames.columns), frames.count()

(7, 10259434)

In [57]:
frames.show(5,truncate=False)

+------------------------------------+----------------------------------------------------------------------------------------------------------------------------+--------+--------+-----+------+---------------------------------------+
|id                                  |visible_area                                                                                                                |match_id|teammate|actor|keeper|location                               |
+------------------------------------+----------------------------------------------------------------------------------------------------------------------------+--------+--------+-----+------+---------------------------------------+
|25dfc952-7a85-464f-b884-982064d46cc9|[82.05880352279489, 80.0, 37.469993200629204, 80.0, 3.833501476649669, 0.0, 115.5840010749895, 0.0, 82.05880352279489, 80.0]|3942819 |true    |false|true  |[22.697541663925428, 41.01277368603656]|
|25dfc952-7a85-464f-b884-982064d46cc9|[82.05880352279489, 80

In [62]:
# Average number of frames per match_id
frames.groupBy('match_id').count().agg({'count':'avg'}).show()

+----------+
|avg(count)|
+----------+
|  51297.17|
+----------+



In [70]:
frames.select('match_id','teammate','actor','location').show(20,truncate=False)

+--------+--------+-----+----------------------------------------+
|match_id|teammate|actor|location                                |
+--------+--------+-----+----------------------------------------+
|3942819 |true    |false|[22.697541663925428, 41.01277368603656] |
|3942819 |true    |false|[46.54665402564941, 46.65992069066397]  |
|3942819 |true    |false|[47.44522057974756, 64.52929870627902]  |
|3942819 |true    |false|[48.81954571089864, 35.810976155696935] |
|3942819 |true    |false|[49.74595437091587, 27.859841158806844] |
|3942819 |true    |false|[53.468934094140124, 48.359720135671516]|
|3942819 |true    |false|[59.39295936892726, 65.20967497250402]  |
|3942819 |false   |false|[59.51839890069241, 50.247564716443506] |
|3942819 |true    |false|[59.63379273609168, 72.56790108034444]  |
|3942819 |true    |false|[59.77039159452952, 29.092900233218515] |
|3942819 |true    |true |[60.0, 40.0]                            |
|3942819 |true    |false|[60.64836725515984, 60.46506018214295